In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import torch.nn as nn
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import IA3Config, get_peft_model, TaskType
from tqdm import tqdm
from matplotlib import pyplot as plt

In [2]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer

In [3]:
from huggingface_hub import login
login()

In [4]:
import re
text = """
Layer 0:
  smallest 1: expert IDs = [2]
  smallest 2: expert IDs = [2, 0]
  smallest 3: expert IDs = [2, 0, 4]
  smallest 4: expert IDs = [2, 0, 4, 7]
Layer 1:
  smallest 1: expert IDs = [1]
  smallest 2: expert IDs = [1, 0]
  smallest 3: expert IDs = [1, 0, 6]
  smallest 4: expert IDs = [1, 0, 6, 4]
Layer 2:
  smallest 1: expert IDs = [4]
  smallest 2: expert IDs = [4, 6]
  smallest 3: expert IDs = [4, 6, 2]
  smallest 4: expert IDs = [4, 6, 2, 0]
Layer 3:
  smallest 1: expert IDs = [3]
  smallest 2: expert IDs = [3, 4]
  smallest 3: expert IDs = [3, 4, 2]
  smallest 4: expert IDs = [3, 4, 2, 0]
Layer 4:
  smallest 1: expert IDs = [4]
  smallest 2: expert IDs = [4, 5]
  smallest 3: expert IDs = [4, 5, 2]
  smallest 4: expert IDs = [4, 5, 2, 6]
Layer 5:
  smallest 1: expert IDs = [3]
  smallest 2: expert IDs = [3, 4]
  smallest 3: expert IDs = [3, 4, 6]
  smallest 4: expert IDs = [3, 4, 6, 2]
Layer 6:
  smallest 1: expert IDs = [3]
  smallest 2: expert IDs = [3, 4]
  smallest 3: expert IDs = [3, 4, 0]
  smallest 4: expert IDs = [3, 4, 0, 6]
Layer 7:
  smallest 1: expert IDs = [3]
  smallest 2: expert IDs = [3, 1]
  smallest 3: expert IDs = [3, 1, 2]
  smallest 4: expert IDs = [3, 1, 2, 4]
Layer 8:
  smallest 1: expert IDs = [0]
  smallest 2: expert IDs = [0, 3]
  smallest 3: expert IDs = [0, 3, 5]
  smallest 4: expert IDs = [0, 3, 5, 4]
Layer 9:
  smallest 1: expert IDs = [6]
  smallest 2: expert IDs = [6, 1]
  smallest 3: expert IDs = [6, 1, 0]
  smallest 4: expert IDs = [6, 1, 0, 4]
Layer 10:
  smallest 1: expert IDs = [6]
  smallest 2: expert IDs = [6, 7]
  smallest 3: expert IDs = [6, 7, 1]
  smallest 4: expert IDs = [6, 7, 1, 3]
Layer 11:
  smallest 1: expert IDs = [5]
  smallest 2: expert IDs = [5, 0]
  smallest 3: expert IDs = [5, 0, 6]
  smallest 4: expert IDs = [5, 0, 6, 1]
Layer 12:
  smallest 1: expert IDs = [5]
  smallest 2: expert IDs = [5, 6]
  smallest 3: expert IDs = [5, 6, 1]
  smallest 4: expert IDs = [5, 6, 1, 0]
Layer 13:
  smallest 1: expert IDs = [5]
  smallest 2: expert IDs = [5, 0]
  smallest 3: expert IDs = [5, 0, 4]
  smallest 4: expert IDs = [5, 0, 4, 7]
Layer 14:
  smallest 1: expert IDs = [5]
  smallest 2: expert IDs = [5, 0]
  smallest 3: expert IDs = [5, 0, 3]
  smallest 4: expert IDs = [5, 0, 3, 7]
Layer 15:
  smallest 1: expert IDs = [1]
  smallest 2: expert IDs = [1, 3]
  smallest 3: expert IDs = [1, 3, 5]
  smallest 4: expert IDs = [1, 3, 5, 7]
Layer 16:
  smallest 1: expert IDs = [4]
  smallest 2: expert IDs = [4, 5]
  smallest 3: expert IDs = [4, 5, 0]
  smallest 4: expert IDs = [4, 5, 0, 3]
Layer 17:
  smallest 1: expert IDs = [0]
  smallest 2: expert IDs = [0, 2]
  smallest 3: expert IDs = [0, 2, 7]
  smallest 4: expert IDs = [0, 2, 7, 1]
Layer 18:
  smallest 1: expert IDs = [4]
  smallest 2: expert IDs = [4, 3]
  smallest 3: expert IDs = [4, 3, 2]
  smallest 4: expert IDs = [4, 3, 2, 1]
Layer 19:
  smallest 1: expert IDs = [3]
  smallest 2: expert IDs = [3, 4]
  smallest 3: expert IDs = [3, 4, 2]
  smallest 4: expert IDs = [3, 4, 2, 7]
Layer 20:
  smallest 1: expert IDs = [7]
  smallest 2: expert IDs = [7, 3]
  smallest 3: expert IDs = [7, 3, 2]
  smallest 4: expert IDs = [7, 3, 2, 6]
Layer 21:
  smallest 1: expert IDs = [1]
  smallest 2: expert IDs = [1, 6]
  smallest 3: expert IDs = [1, 6, 5]
  smallest 4: expert IDs = [1, 6, 5, 3]
Layer 22:
  smallest 1: expert IDs = [3]
  smallest 2: expert IDs = [3, 1]
  smallest 3: expert IDs = [3, 1, 4]
  smallest 4: expert IDs = [3, 1, 4, 7]
Layer 23:
  smallest 1: expert IDs = [1]
  smallest 2: expert IDs = [1, 4]
  smallest 3: expert IDs = [1, 4, 5]
  smallest 4: expert IDs = [1, 4, 5, 2]
Layer 24:
  smallest 1: expert IDs = [6]
  smallest 2: expert IDs = [6, 5]
  smallest 3: expert IDs = [6, 5, 4]
  smallest 4: expert IDs = [6, 5, 4, 1]
Layer 25:
  smallest 1: expert IDs = [5]
  smallest 2: expert IDs = [5, 2]
  smallest 3: expert IDs = [5, 2, 4]
  smallest 4: expert IDs = [5, 2, 4, 1]
Layer 26:
  smallest 1: expert IDs = [6]
  smallest 2: expert IDs = [6, 4]
  smallest 3: expert IDs = [6, 4, 1]
  smallest 4: expert IDs = [6, 4, 1, 5]
Layer 27:
  smallest 1: expert IDs = [0]
  smallest 2: expert IDs = [0, 7]
  smallest 3: expert IDs = [0, 7, 1]
  smallest 4: expert IDs = [0, 7, 1, 5]
Layer 28:
  smallest 1: expert IDs = [6]
  smallest 2: expert IDs = [6, 5]
  smallest 3: expert IDs = [6, 5, 1]
  smallest 4: expert IDs = [6, 5, 1, 4]
Layer 29:
  smallest 1: expert IDs = [7]
  smallest 2: expert IDs = [7, 4]
  smallest 3: expert IDs = [7, 4, 0]
  smallest 4: expert IDs = [7, 4, 0, 1]
Layer 30:
  smallest 1: expert IDs = [3]
  smallest 2: expert IDs = [3, 1]
  smallest 3: expert IDs = [3, 1, 6]
  smallest 4: expert IDs = [3, 1, 6, 0]
Layer 31:
  smallest 1: expert IDs = [4]
  smallest 2: expert IDs = [4, 0]
  smallest 3: expert IDs = [4, 0, 6]
  smallest 4: expert IDs = [4, 0, 6, 3]
"""

k_dicts = {1: {}, 2: {}, 3: {}, 4: {}}

current_layer = None

for line in text.split("\n"):
    line = line.strip()
    
    m_layer = re.match(r"Layer\s+(\d+):", line)
    if m_layer:
        current_layer = int(m_layer.group(1))
        continue
    
    m_k = re.match(r"smallest\s+(\d+):\s+expert IDs\s*=\s*\[([0-7,\s]*)\]", line)
    if m_k:
        k = int(m_k.group(1))
        ids = m_k.group(2)
        ids = ids.strip()
        if ids == "":
            parsed = []
        else:
            parsed = [int(x) for x in ids.split(",")]
        k_dicts[k][current_layer] = parsed

for k in range(1, 5):
    print(f"\n===== k = {k} =====")
    for layer in range(32):
        print(layer, ":", k_dicts[k][layer])



===== k = 1 =====
0 : [2]
1 : [1]
2 : [4]
3 : [3]
4 : [4]
5 : [3]
6 : [3]
7 : [3]
8 : [0]
9 : [6]
10 : [6]
11 : [5]
12 : [5]
13 : [5]
14 : [5]
15 : [1]
16 : [4]
17 : [0]
18 : [4]
19 : [3]
20 : [7]
21 : [1]
22 : [3]
23 : [1]
24 : [6]
25 : [5]
26 : [6]
27 : [0]
28 : [6]
29 : [7]
30 : [3]
31 : [4]

===== k = 2 =====
0 : [2, 0]
1 : [1, 0]
2 : [4, 6]
3 : [3, 4]
4 : [4, 5]
5 : [3, 4]
6 : [3, 4]
7 : [3, 1]
8 : [0, 3]
9 : [6, 1]
10 : [6, 7]
11 : [5, 0]
12 : [5, 6]
13 : [5, 0]
14 : [5, 0]
15 : [1, 3]
16 : [4, 5]
17 : [0, 2]
18 : [4, 3]
19 : [3, 4]
20 : [7, 3]
21 : [1, 6]
22 : [3, 1]
23 : [1, 4]
24 : [6, 5]
25 : [5, 2]
26 : [6, 4]
27 : [0, 7]
28 : [6, 5]
29 : [7, 4]
30 : [3, 1]
31 : [4, 0]

===== k = 3 =====
0 : [2, 0, 4]
1 : [1, 0, 6]
2 : [4, 6, 2]
3 : [3, 4, 2]
4 : [4, 5, 2]
5 : [3, 4, 6]
6 : [3, 4, 0]
7 : [3, 1, 2]
8 : [0, 3, 5]
9 : [6, 1, 0]
10 : [6, 7, 1]
11 : [5, 0, 6]
12 : [5, 6, 1]
13 : [5, 0, 4]
14 : [5, 0, 3]
15 : [1, 3, 5]
16 : [4, 5, 0]
17 : [0, 2, 7]
18 : [4, 3, 2]
19 : [3, 4, 2]
2

In [5]:
import torch
import torch.nn as nn
from transformers import MixtralForCausalLM, AutoTokenizer
import os
import shutil
from pathlib import Path

# GPU and cache settings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["HF_HOME"] = "/workspace/hf_cache"

In [6]:
# empty the cache
torch.cuda.empty_cache()

In [7]:
def check_disk_space(path, required_gb=100):
	"""Check if there's enough disk space available."""
	stats = shutil.disk_usage(path)
	available_gb = stats.free / (2**30)  # Convert to GB
	return available_gb >= required_gb, available_gb

class PrunedMixtralSparseMoeBlock(nn.Module):
	def __init__(self, original_moe, pruned_experts):
		super().__init__()
		self.pruned_experts = sorted(pruned_experts)
		
		# Prune the gate network
		with torch.no_grad():
			original_weight = original_moe.gate.weight
			mask = torch.ones(original_weight.size(0), dtype=torch.bool)
			mask[pruned_experts] = False
			new_weight = original_weight[mask]

		self.gate = nn.Linear(original_moe.gate.in_features, original_moe.gate.out_features - len(pruned_experts), bias=False)
		self.gate = self.gate.to(original_weight.device)
		self.gate.weight.data = new_weight
		
		# Actually remove the pruned experts
		self.experts = nn.ModuleList([expert for i, expert in enumerate(original_moe.experts) if i not in pruned_experts])
		self.num_experts = len(self.experts)

	def forward(self, hidden_states):
		gate_logits = self.gate(hidden_states)
		weights, selected_experts = torch.topk(gate_logits, k=2, dim=-1)
		weights = nn.functional.softmax(weights, dim=-1)
		
		hidden_states = hidden_states.unsqueeze(1)  # Add sequence length dimension
		expert_outputs = torch.zeros_like(hidden_states)
		for i, expert in enumerate(self.experts):
			expert_mask = (selected_experts == i).any(dim=-1).unsqueeze(-1)
			expert_inputs = hidden_states * expert_mask
			expert_outputs += expert(expert_inputs) * expert_mask
		
		hidden_states = hidden_states.squeeze(1)  # Remove sequence length dimension
		expert_outputs = expert_outputs.squeeze(1)
		
		output = torch.einsum("...e,...ec->...c", weights, expert_outputs)
		return output

def prune_mixtral_experts(model, pruned_experts_per_layer):
	for layer_idx, pruned_experts in pruned_experts_per_layer.items():
		original_moe = model.model.layers[layer_idx].block_sparse_moe
		pruned_moe = PrunedMixtralSparseMoeBlock(original_moe, pruned_experts)
		model.model.layers[layer_idx].block_sparse_moe = pruned_moe
	return model

In [8]:
# Check disk space in the HF cache directory
# cache_dir = "/root/.cache/huggingface"
cache_dir = "/root/.cache/huggingface/"
has_space, available_gb = check_disk_space(cache_dir)

if not has_space:
    print(f"Warning: Only {available_gb:.2f}GB available in {cache_dir}")
    response = input("Continue anyway? (y/n): ")
    if response.lower() != 'y':
        print("Aborting operation.")

In [9]:
print("Loading model...")
model = MixtralForCausalLM.from_pretrained(
    "mistralai/Mixtral-8x7B-Instruct-v0.1",
    torch_dtype=torch.float16,  # Use half precision
    device_map="auto"  # Automatically handle multi-GPU
    # device_map=None,
    # low_cpu_mem_usage=False
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading model...


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
for k in [1]:
    print(f"----------------------prune {k} experts---------------------")
    temp_save_dir = Path(f"./temp_model_save_{k}")
    temp_save_dir.mkdir(exist_ok=True)

    print("Loading model...")
    model = MixtralForCausalLM.from_pretrained(
        "mistralai/Mixtral-8x7B-Instruct-v0.1",
        torch_dtype=torch.float16,  # Use half precision
        # device_map="auto"  # Automatically handle multi-GPU
        device_map=None,
        low_cpu_mem_usage=False
    )
    
    pruned_experts_per_layer = k_dicts[k]
    print(pruned_experts_per_layer)

    print("Pruning the model...")
    pruned_model = prune_mixtral_experts(model, pruned_experts_per_layer)
    print("Model pruned!")

    for n, p in pruned_model.named_parameters():
    if getattr(p, "is_meta", False):
        print("META FOUND!!!!!!!!!!!!:", n)
    print(pruned_model)
    
    # Save in chunks first
    print(f"Saving model to temporary directory: {temp_save_dir}")
    pruned_model.save_pretrained(
        temp_save_dir,
        max_shard_size="2GB",
        safe_serialization=True
    )
    tokenizer.save_pretrained(temp_save_dir)

    print("Pushing to Hugging Face Hub...")
    pruned_model.push_to_hub(
        repo_id=f"kaaiiii/Mixtral_prune_{k}_experts",
        max_shard_size="2GB",
        safe_serialization=True
    )
    tokenizer.push_to_hub(
        f"kaaiiii/Mixtral_prune_{k}_experts",
    )
    print("Successfully pushed to Hub!")

    

----------------------prune 1 experts---------------------
Loading model...


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


{0: [2], 1: [1], 2: [4], 3: [3], 4: [4], 5: [3], 6: [3], 7: [3], 8: [0], 9: [6], 10: [6], 11: [5], 12: [5], 13: [5], 14: [5], 15: [1], 16: [4], 17: [0], 18: [4], 19: [3], 20: [7], 21: [1], 22: [3], 23: [1], 24: [6], 25: [5], 26: [6], 27: [0], 28: [6], 29: [7], 30: [3], 31: [4]}
Pruning the model...
Model pruned!
MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (block_sparse_moe): PrunedMixtralSparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=7, bias=False)
          (experts): ModuleList(
            (0-6):

/workspace/MoE_pruning/venv/lib/python3.12/site-packages/transformers/modeling_utils.py:3970: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/42 [00:00<?, ?it/s]

NotImplementedError: Cannot copy out of meta tensor; no data!